# Import necessary depencencies

In [ ]:
import numpy as np
from keras.layers import Dropout, Activation, Dense
from keras.models import Sequential
import model_evaluation_utils as meu
import utils
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM
import xgboost as xgb
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions

np.set_printoptions(precision=2, linewidth=80)


# Load normalized data

In [ ]:
# train and test datasets
train_reviews = utils.readFromDisk('train_reviews')
train_sentiments = utils.readFromDisk('train_sentiments')
test_reviews = utils.readFromDisk('test_reviews')
test_sentiments = utils.readFromDisk('test_sentiments')
tokenized_train = utils.readFromDisk('tokenized_train')
tokenized_test = utils.readFromDisk('tokenized_test')
train_sentiments_encoded = utils.readFromDisk('train_sentiments_encoded')
test_sentiments_encoded = utils.readFromDisk('test_sentiments_encoded')

#glove features
# feature engineering with GloVe model
train_glove_features = utils.readFromDisk('train_glove_features')
test_glove_features = utils.readFromDisk('test_glove_features')

# Logistic Regression

In [ ]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

lr = LogisticRegression(penalty='l2', max_iter=500, C=1)

## Logistic Regression - Training and Prediction

In [ ]:
# Logistic Regression model
lr_predictions = meu.train_predict_model(classifier=lr, 
                                             train_features=train_glove_features, train_labels=train_sentiments,
                                             test_features=test_glove_features, test_labels=test_sentiments)

## Logistic Regression - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=lr_predictions,
                                      classes=[1,0])

# SVM 

In [ ]:
svm = SGDClassifier(loss='hinge', max_iter=500)


## SVM - Training and Prediction

In [ ]:
svm_predictions = meu.train_predict_model(classifier=svm, 
                                             train_features=train_glove_features, train_labels=train_sentiments,
                                             test_features=test_glove_features, test_labels=test_sentiments)

## SVM - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=svm_predictions,
                                      classes=[1,0])

# Deep neural network architecture (DNN)

In [ ]:
def construct_deepnn_architecture(num_input_features):
    dnn_model = Sequential()
    dnn_model.add(Dense(512, activation='relu', input_shape=(num_input_features,)))
    dnn_model.add(Dropout(0.2))
    dnn_model.add(Dense(512, activation='relu'))
    dnn_model.add(Dropout(0.2))
    dnn_model.add(Dense(512, activation='relu'))
    dnn_model.add(Dropout(0.2))
    dnn_model.add(Dense(2))
    dnn_model.add(Activation('softmax'))

    dnn_model.compile(loss='categorical_crossentropy', optimizer='adam',                 
                      metrics=['accuracy'])
    return dnn_model



## DNN - Training and Prediction

In [ ]:
dnn = construct_deepnn_architecture(num_input_features=300)

try:   
    dnn=utils.readFromDisk('dnn')
except FileNotFoundError as te:    
    batch_size = 100
    dnn.fit(train_glove_features, train_sentiments_encoded, epochs=5, batch_size=batch_size, 
            shuffle=True, validation_split=0.1, verbose=1)
    utils.writeToDisk(dnn,'dnn')


In [ ]:
le = utils.readFromDisk('label_encoder')
y_pred = dnn.predict_classes(test_glove_features)
dnn_predictions = le.inverse_transform(y_pred) 

## DNN - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=dnn_predictions, 
                                      classes=[1,0])  


# XGBoost

In [ ]:
xg = xgb.XGBClassifier(n_estimators=500, max_depth=5, base_score=0.5,objective='binary:logistic', random_state=42,nthread=16)


## XGBoost - Training and Prediction

In [ ]:
try:   
    xg=utils.readFromDisk('xg')
except FileNotFoundError as te:    
    xg.fit(train_glove_features, train_sentiments)
    utils.writeToDisk(xg,'xg')


In [ ]:
xg_predictions= xg.predict(test_glove_features)


## XGBoost - Performance


In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=xg_predictions, 
                                      classes=[1,0])  


# LSTM


In [ ]:
from collections import Counter

# build word to index vocabulary
token_counter = Counter([token for review in tokenized_train for token in review])
vocab_map = {item[0]: index+1 for index, item in enumerate(dict(token_counter).items())}
max_index = np.max(list(vocab_map.values()))
vocab_map['PAD_INDEX'] = 0
vocab_map['NOT_FOUND_INDEX'] = max_index+1
vocab_size = len(vocab_map)
# view vocabulary size and part of the vocabulary map
print('Vocabulary Size:', vocab_size)
print('Sample slice of vocabulary map:', dict(list(vocab_map.items())[10:20]))

## LSTM- Encode and Pad datasets & Encode prediction class labels

In [ ]:
# get max length of train corpus and initialize label encoder
max_len = np.max([len(review) for review in tokenized_train])

## Train reviews data corpus
# Convert tokenized text reviews to numeric vectors
train_X = [[vocab_map[token] for token in tokenized_review] for tokenized_review in tokenized_train]
train_X = sequence.pad_sequences(train_X, maxlen=max_len) # pad 

## Test reviews data corpus
# Convert tokenized text reviews to numeric vectors
test_X = [[vocab_map[token] if vocab_map.get(token) else vocab_map['NOT_FOUND_INDEX'] 
           for token in tokenized_review] 
              for tokenized_review in tokenized_test]
test_X = sequence.pad_sequences(test_X, maxlen=max_len)

# view vector shapes
print('Max length of train review vectors:', max_len)
print('Train review vectors shape:', train_X.shape, ' Test review vectors shape:', test_X.shape)

## LSTM- Build, train and visualize the LSTM Model

In [ ]:
EMBEDDING_DIM = 128 # dimension for dense embeddings for each token
LSTM_DIM = 64 # total LSTM units

lstm = Sequential()

try:   
    lstm=utils.readFromDisk('lstm')
    print(lstm.summary())
except FileNotFoundError as te:    
    lstm.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=max_len))
    lstm.add(SpatialDropout1D(0.2))
    lstm.add(LSTM(LSTM_DIM, dropout=0.2, recurrent_dropout=0.2))
    lstm.add(Dense(2, activation="sigmoid"))
    
    lstm.compile(loss="binary_crossentropy", optimizer="adam",
                  metrics=["accuracy"])
    print(lstm.summary())

    SVG(model_to_dot(lstm, show_shapes=True, show_layer_names=False, 
                     rankdir='LR').create(prog='dot', format='svg'))
    batch_size = 100
    lstm.fit(train_X, train_sentiments_encoded, epochs=2, batch_size=batch_size, 
              shuffle=True, validation_split=0.1, verbose=1)

    utils.writeToDisk(lstm,'lstm')


## LSTM - Predictions

In [ ]:
lstm_pred_test = lstm.predict_classes(test_X)

In [ ]:
lstm_predictions = le.inverse_transform(lstm_pred_test)

## LSTM - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=lstm_predictions, 
                                      classes=[1,0])  

# Ensemble Stacking models

In [ ]:
clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)


## Ensemble Stacking - n fold validations

In [ ]:
clf_list = [clf1, clf2, clf3, sclf]
scores_list=[]

try:
    clf_list=utils.readFromDisk('clf_list');
    scores_list=utils.readFromDisk('scores_list');
except FileNotFoundError as te:    
    for clf in clf_list:
        scores = cross_val_score(clf, train_glove_features, train_sentiments, cv=3, scoring='accuracy',n_jobs=-1)
        scores_list.append(scores)
    utils.writeToDisk(clf_list,'clf_list')
    utils.writeToDisk(scores_list,'scores_list')

## Ensemble Stacking - metrics and visualizations


In [ ]:
label = ['KNN', 'Random Forest', 'Naive Bayes', 'Stacking Classifier']

clf_cv_mean = []
clf_cv_std = []

fig = plt.figure(figsize=(10,8))
gs = gridspec.GridSpec(2, 2)
grid = itertools.product([0,1],repeat=2)

for clf, label, grd, scores in zip(clf_list, label, grid,scores_list):


    print ("Accuracy: %.2f (+/- %.2f) [%s]" %(scores.mean(), scores.std(), label))
    clf_cv_mean.append(scores.mean())
    clf_cv_std.append(scores.std())
        
    clf.fit(train_glove_features, train_sentiments)
    ax = plt.subplot(gs[grd[0], grd[1]])
    fig = plot_decision_regions(X=train_glove_features, y=train_sentiments, clf=clf)
    plt.title(label)

plt.show()     




# All Models Evaluation

## Model ROC curves

In [ ]:
plt.figure(0).clf()

color = ['blue', 'orange', 'red', 'green', 'coral',
             'grey', 'indigo', 'gold', 'lime', 'olive',
             'pink', 'navy', 'magenta', 'yellow', 'tomato',
             'turquoise', 'yellowgreen', 'maroon', 'lightblue']
mlr=[]
msvm=[]
mdnn=[]
mxg=[]
mlstm=[]

def metricsAndROC(pred,metricsArray,rocTitle,colorIndex):
    fpr, tpr, thresholds = metrics.roc_curve(test_sentiments, pred)
    auc = metrics.roc_auc_score(test_sentiments, pred)
    metricsArray.append(metrics.f1_score(test_sentiments, pred))
    metricsArray.append(metrics.precision_score(test_sentiments, pred))
    metricsArray.append(metrics.accuracy_score(test_sentiments, pred))
    metricsArray.append(metrics.recall_score(test_sentiments, pred))
    
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr,color=color[colorIndex], label=rocTitle.format(auc))



metricsAndROC(lr_predictions,mlr,'LR',0)
metricsAndROC(svm_predictions,msvm,'SVM',1)
metricsAndROC(svm_predictions,mdnn,'DNN',2)
metricsAndROC(xg_predictions,mxg,'XGBoost',3)
metricsAndROC(lstm_predictions,mlstm,'LSTM',4)



#show the roc curve now
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
# show the legend
plt.legend(loc='best')
plt.show()

## All Models Metrics comparison

In [ ]:
n_groups = 4
index = np.arange(n_groups)
bar_width = .1

plt.bar(index,mlr, bar_width, color=color[0], label='LR')


z=index + bar_width
plt.bar(z, msvm, bar_width, color=color[1],label='SVM')


z=z+ bar_width
plt.bar(z, mdnn, bar_width, color=color[2], label='DNN')

z=z+ bar_width
plt.bar(z,mxg , bar_width, color=color[3], label='XGB')

z=z+ bar_width
plt.bar(z,mlstm , bar_width,color=color[4], label='LSTM')



#ax.set_xlabel('Metric')
#ax.set_ylabel('Value')
#ax.set_title('Comparison of Feature Engineering Models on Amazon Reviews')
#ax.set_xticks(index + bar_width / 2)
pltLabels=['F1','PRECISION','ACCURACY','RECALL']
# Add xticks on the middle of the group bars
plt.xlabel('Model Metrics', fontweight='bold')
plt.xticks([r + bar_width for r in range(n_groups)], pltLabels)
 
# Create legend & Show graphic
plt.legend(frameon=False,ncol=3, loc='lower left')
plt.show()
